In [1]:
import requests
import pandas as pd

In [2]:
baseUrl = "https://api.themoviedb.org/"
apiKey = "API-KEY"
movieRequestRoute = "3/movie/"

In [3]:
xl = pd.read_excel('blank-check-data.xlsx', sheet_name=None, index_col='CODE')
df_miniseries = xl['MINISERIES']
df_films = xl['FILMS']
df_films.head()

,NAME,MINISERIES,PATREON,NONDIRECTOR,RELEASEDATE,BUDGET,REVENUE,RUNTIME,MINISERIESNAME
CODE,,,,,,,,,
1893,Star Wars: Episode I - The Phantom Menace,STARWARS,False,NaN,1999-05-19,115000000,924317558,136.0,Star Wars
205587,The Judge,JUDGE,False,NaN,2014-10-08,50000000,83719388,141.0,Judging the Judge
1894,Star Wars: Episode II - Attack of the Clones,STARWARS,False,NaN,2002-05-15,120000000,649398328,142.0,Star Wars
22059,The Fantastic Four,FANTASTICFOUR,False,NaN,1994-01-01,1500000,0,90.0,The Podcastic Two
9738,Fantastic Four,FANTASTICFOUR,False,NaN,2005-06-29,100000000,330579719,106.0,The Podcastic Two


In [4]:
def getMovieByExternalId(externalId) :
    url = baseUrl + movieRequestRoute + str(externalId) + "?api_key=" + apiKey
    print("getting info for film with externalId:", externalId)
    r = requests.get(url)
    return r.json()

In [5]:
def getProductionCountries(film):
    return film['production_countries']

In [6]:
films_production_countries = []

for (i, val) in df_films.iterrows():
    film = getMovieByExternalId(i)
    
    miniseriesName = df_miniseries.loc[val['MINISERIES'],'NAME']
    filmName = val['NAME']
    patreon = val['PATREON']
    
    for country in getProductionCountries(film):
        films_production_countries.append({
            'ISO_CODE': country['iso_3166_1'],
            'COUNTRY': country['name'],
            'FILM': filmName,
            'MINISERIES': miniseriesName,
            'PATREON': patreon
        })

df_films_production_countries = pd.DataFrame(films_production_countries)
df_films_production_countries.head()

getting info for film with externalId: 1893


KeyError: 'production_countries'

In [ ]:
import matplotlib.pyplot as plt

count_by_country = [{'COUNTRY': country, 'FILMCOUNT': len(film_group), 'FILMS': [film['FILM'] for index, film in film_group.iterrows()], 'CODE': film_group.iloc[0]['ISO_CODE']} for country, film_group in df_films_production_countries.groupby(['COUNTRY'])]
df_count_by_country = pd.DataFrame(count_by_country).sort_values('FILMCOUNT', ascending=False)
df_count_by_country.head()

In [ ]:
print("Film count", len(df_films))
print("Country count:", len(df_count_by_country))

In [ ]:
df_count_by_country.plot(kind='bar', y='FILMCOUNT', x='COUNTRY')
plt.show()

In [ ]:
for key, country in df_count_by_country.iterrows():
    print("**" + str(country['FILMCOUNT']) + "**. *" + country['COUNTRY'] + "*  ")
    filmNames = ", ".join(country['FILMS'][:25])
    if len(country['FILMS']) > 25:
        filmNames += ",..."
    print(filmNames + "  ")

In [ ]:
import folium
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
countries_geo = f'{url}/world-countries.json'

bins = list(df_count_by_country['FILMCOUNT'].quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(location=[0, 0], zoom_start=2)

folium.Choropleth(
    geo_data=countries_geo,
    name='choropleth',
    data=df_count_by_country,
    columns=['COUNTRY', 'FILMCOUNT'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    nan_fill_color='white',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins,
    #legend_name='# FILMS'
).add_to(m)
folium.LayerControl().add_to(m)

m